In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from biclfs_training import main

In [5]:
main()

Table Level Loop:   0%|          | 0/6 [00:00<?, ?it/s]

Fold Loop: 0it [00:00, ?it/s]

HP Level Loop:   0%|          | 0/12 [00:00<?, ?it/s]

Preparing training examples.:   0%|          | 0/18 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1129 > 1024). Running this sequence through the model will result in indexing errors


...

Predicting.:   0%|          | 0/18 [00:00<?, ?it/s]

Evaluating.: 0it [00:00, ?it/s]

Predicting.:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating.: 0it [00:00, ?it/s]

TypeError: log() got an unexpected keyword argument 'group'